## Imports

In [1]:
using HDF5

## Functions

### General

In [ ]:
path = "data/wildfire"

In [ ]:
files = readdir(path)
nFiles = length(files)

In [ ]:
for i = nFiles:0
    @show i
end

In [ ]:
df = h5open("data/wildfire/state_40.hdf5", "r") do file
    read(file, "dataset_1")
end

In [ ]:

df[CartesianIndex(Tuple([1,1]))]

In [ ]:
zeros(Int64,length([2]))

In [ ]:
push!((),[(1,2) false])

In [ ]:
A = [[1,2],[3,4]];
for i in CartesianIndices(A)
    @show i
end

In [2]:
"""
    generateMaskExpr(size)

Creates a string that represents a coroutine that emits all the relative coordinates
from a square neighbourhood 

"""
function generateMaskExpr(size)
    literals = []
    ranges = []
    for ind in 1:length(size)
        @show ind
        charCode = Int('a')+ind
        push!(literals,"$(Char(charCode))")
        push!(ranges,"$(Char(charCode))= -$(size[ind]):$(size[ind])")
    end
    expr1 = join(literals," ")
    expr2 = join(ranges,",")
    expr3 = "function emitMaskElement(chnl::Channel) for $(expr2) put!(chnl,[$(expr1)]) end end"
end

function getMaskTask(size)
    expr = generateMaskExpr(size)
    fun = Meta.parse(expr)
    eval(fun)
end

function isValidCoord(coord,msize)
    isValid = true
    for i in eachindex(coord)
        isValid = isValid && coord[i] >= 1 && coord[i] <= msize[i]
    end
    isValid
end

function manhattanDistance(coord)
    res = 0
    for i in coord
        res += abs(i)
    end
    res
end

function vonNeumannFilter(distance)
    (coord) -> manhattanDistance(coord) <= distance
end

function mooreFilter()
    (coord) -> true
end

function getNeighbourhoodValues(df,coord,maskTask,filter,msize,paddingType,paddingValue)
    neighbourhood = []
    j = 1
    maskChannel = Channel(maskTask)
    for val in maskChannel
        if !filter(val)
            continue
        end
        relCoord = zeros(Int16,length(coord))
        for i = 1:length(coord)
            relCoord[i] = val[i]+coord[i]
        end
        validCoord = isValidCoord(relCoord,msize)
        if !validCoord
            if paddingType == 1
                push!(neighbourhood, paddingValue)
            elseif paddingType == 2
                push!(neighbourhood, 0)
            end
        else
            index = CartesianIndex(Tuple(relCoord))
            push!(neighbourhood, df[index])
        end
        j += 1
    end
    neighbourhood
end


"""
    getNeighborhood(dataframe,mask,paddingType,paddingValue)

Obtains the value of each neighborhood of the dataframe

"""

function getNeighbourhood(df,maskTask,filter,paddingType,paddingValue)
    neighbourhoods = []
    cls = []
    msize = size(df)
    for coord in CartesianIndices(df) 
        push!(neighbourhoods,getNeighbourhoodValues(df,coord,maskTask,filter,msize,paddingType,paddingValue))
    end
    neighbourhoods
end


function getCls(df)
    cls = []
    for coord in CartesianIndices(df) 
        push!(cls,Int(df[coord]))
    end
    cls
end

"""
    problemTask(states,neighborhood,paddingType,paddingValue)

Creates a task that returns in each execution the values of the neihgborhood

"""
function problemTask(c::Channel,folder,states,maskTask,filter,paddingType,paddingValue)
    prev = h5open("$(folder)/state_$(states[1]).hdf5", "r") do file
                read(file, "dataset_1")
            end
    
    for i = 2:length(states)
        next = h5open("$(folder)/state_$(states[i]).hdf5", "r") do file
                    read(file, "dataset_1")
                end
        neighbourhood = getNeighbourhood(prev,maskTask,filter,paddingType,paddingValue)
        cls = getCls(next)
        itm = "$(folder)/state_$(states[i]).hdf5",neighbourhood,cls
        prev = next
        put!(c,itm)
        @show i
    end
    
end

problemTask

### Genetic

In [ ]:
"""c = Channel((c) -> problemTask(c,"data/square2",[0 1 30 31 40 41],[[-1,-1],[0,0],[1,1]],1,1))"""
n = 1
"""taskHdl = Task(() -> problemTask("data/wildfire",[0 1 2 3 4],[[-1,-1],[0,0],[1,1]],1,1))
"""


## Tests

In [24]:
dataPath = "data/square2"
radious = [3]
maskTask = getMaskTask(radious)
filter = mooreFilter()
paddingType = 1
paddingValue = -1

ind = 1


-1

In [21]:
@time c = Channel((c) -> problemTask(c,dataPath,[0 1],maskTask,filter,paddingType,paddingValue))

  0.071262 seconds (96.54 k allocations: 5.571 MiB, 11.69% gc time)


Channel{Any}(sz_max:0,sz_curr:0)

In [3]:
function mytask(c::Channel,myarg)
    for i = 1:myarg
        @show i
        put!(c,i)
    end
end

b = Channel(1)
taskHdl = Task(() -> mytask(b,7))
# or, equivalently
taskHdl = @task mytask(7)

Task (runnable) @0x0000000115e3d8d0

In [4]:
bind(b,taskHdl)

Channel{Any}(sz_max:1,sz_curr:0)

In [ ]:
for i in b
    @show i
end

In [ ]:
open("hello.txt") do f
       uppercase(read(f, String))
   end